In [1]:
import pandas as pd
import datetime
import os
import numpy as np
import itertools
os.chdir('../')
from configs.keylog_cfg import *

In [2]:
files_to_read = [READ_KEYLOGGED_DIR + i for i in list(itertools.chain(*list(itertools.chain.from_iterable([f for f in os.walk(READ_KEYLOGGED_DIR)])))) if i.endswith('.txt')]
files_to_read

['keylogs/raw_keylogs/keylog_2021-11-2019:48:40.189655.txt',
 'keylogs/raw_keylogs/keylog_2021-11-2018:44:51.722410.txt',
 'keylogs/raw_keylogs/keylog_2021-11-2018:49:15.126376.txt']

In [3]:
raw_keylogged_data = []
for f in files_to_read:
    raw_keylogged_data.append(open(f, 'r').readlines())

In [5]:
def raw_keylog_data_to_df(raw_keylogged_data):
    
    list_of_dfs = []

    counter = 0

    for data in raw_keylogged_data:

        tmp = {k:v for k,v in [i.split(': ') for i in data]}
        raw_keylog_dict = {}
        raw_keylog_dict['timestamp'] = list(tmp.keys())
        raw_keylog_dict['keypress'] = list(tmp.values())

        df_tmp = pd.DataFrame(raw_keylog_dict)
        df_tmp['file_suffix'] = files_to_read[counter][-11:].replace('.', '').replace('txt', '')
        counter += 1

        list_of_dfs.append(df_tmp)

    df = pd.concat(list_of_dfs, axis=0)
    
    return df    

In [6]:
df = raw_keylog_data_to_df(raw_keylogged_data)
df['keypress'] = df['keypress'].str.replace('\n', '').str.replace("'", "")
df

,timestamp,keypress,file_suffix
0,"2021-11-20 19:48:40,538",j,189655
1,"2021-11-20 19:48:40,539",k,189655
2,"2021-11-20 19:48:40,562",a,189655
3,"2021-11-20 19:48:40,634",s,189655
4,"2021-11-20 19:48:40,651",l,189655
...,...,...,...
2958,"2021-11-20 19:22:39,664",Key.alt,126376
2959,"2021-11-20 19:22:39,970",2,126376
2960,"2021-11-20 19:22:40,626",Key.alt,126376
2961,"2021-11-20 19:22:41,231",Key.ctrl,126376


In [7]:
df['file_suffix'].value_counts()

126376    2963
722410     673
189655      29
Name: file_suffix, dtype: int64

In [10]:
def group_keylogged_df(df, groupby_keypress = 'Key.enter'):
    
    df['group'] = np.nan
    df.loc[df['keypress'] == groupby_keypress, 'group'] = 1
    df['group'] = df.groupby('keypress')['group'].transform(lambda x: x.cumsum())
    df['group'].bfill(inplace=True)
    
    df_grouped = df.groupby('group').agg({'timestamp': [min, max],
                             'keypress': [lambda x: ''.join(x)],
                             'file_suffix': [min]})
    
    df_grouped.columns = list(pd.Index([str(e[0]).lower() + '_' + str(e[1]).lower()\
                                        for e in df_grouped.columns.tolist()])\
                              .str.replace(' ', '_')\
                              .str.replace('_<lambda>', '')\
                              .str.replace('file_suffix_min', 'file_suffix'))
    df_grouped.reset_index(inplace=True, drop=True)

    return df_grouped

In [12]:
df_tmp = group_keylogged_df(df)
df_tmp

,timestamp_min,timestamp_max,keypress,file_suffix
0,"2021-11-20 18:44:52,683","2021-11-20 19:48:42,125",jkasldfkjaf;askldjf;asdlkfjKey.ctrlcKey.ctrlKe...,189655
1,"2021-11-20 18:44:58,280","2021-11-20 18:44:59,685",facebook.comKey.enter,722410
2,"2021-11-20 18:45:00,582","2021-11-20 18:45:04,497",himyemailKey.shift@yes.comKey.enter,722410
3,"2021-11-20 18:45:05,631","2021-11-20 18:45:08,879",nothisisnotapasswordKey.enter,722410
4,"2021-11-20 18:45:10,478","2021-11-20 18:45:12,588",Key.tabKey.tabKey.ctrlwKey.enter,722410
...,...,...,...,...
229,"2021-11-20 19:22:18,795","2021-11-20 19:22:20,282",kkjKey.enter,126376
230,"2021-11-20 19:22:20,906","2021-11-20 19:22:24,422",Key.downKey.downKey.upKey.upKey.leftKey.backsp...,126376
231,"2021-11-20 19:22:24,920","2021-11-20 19:22:28,017",skjKey.enter,126376
232,"2021-11-20 19:22:28,376","2021-11-20 19:22:31,992","df[""""filKey.tabKey.upKey.downKey.enter",126376


'keylogs/parsed_keylogs/2021-11-2019:53:42.215323'